In [6]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import dask.dataframe as dd
from pyspark.sql import SparkSession
import multiprocessing as mp
import csv
import datatable as dt
import yaml
import gzip
import os
from subprocess import check_call
import warnings
warnings.filterwarnings('ignore')
import time

In [7]:
# Dataframes implement the pandas API
import dask.dataframe as dd
df = pd.read_csv('CA2013.csv')
df.head()

,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,ethnicity
0,CA-2013-0000001,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,Hispanic,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,H
1,CA-2013-0000002,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,F,...,Black,Moving Violation (VC),Moving violation,False,No Search,NaN,False,CHP 215,False,B
2,CA-2013-0000003,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,White,Moving Violation (VC),Moving violation,False,No Search,NaN,False,CHP 215,False,W
3,CA-2013-0000004,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,White,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,W
4,CA-2013-0000005,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,Black,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,B


# Read Data using Pandas

In [8]:
#import pandas as pd
import pandas
pandas.Int64Index
import modin.pandas as mpd
import dask.dataframe as dd
import time

# read using Dask

In [9]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('CA2013.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.1392660140991211 sec


# read using Pandas

In [10]:
import pandas as pd
start = time.time()
df = pd.read_csv('CA2013.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  92.62796807289124 sec


# Dask is better than pandas with 0.134 sec

In [25]:
from dask import dataframe as dd
df = dd.read_csv('CA2013.csv',delimiter=',')

In [26]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 23 entries, id to ethnicity
dtypes: object(16), bool(2), float64(5)

In [27]:
#No. of Rows
len(df.index)

14536338

In [28]:
#No, of Columns
len(df.columns)

23

In [29]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [30]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [31]:
data=df.columns
data

Index(['id', 'state', 'stop_date', 'stop_time', 'location_raw', 'county_name',
       'county_fips', 'fine_grained_location', 'police_department',
       'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race_raw',
       'driver_race', 'violation_raw', 'violation', 'search_conducted',
       'search_type_raw', 'search_type', 'contraband_found', 'stop_outcome',
       'is_arrested', 'ethnicity'],
      dtype='object')

# Validation

In [41]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [33]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [4]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: CA2013
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
      - id
      - State
      - stop_date
      - stop_time
      - location_raw
      - county_name
      - county_fips
      - fine_grained_location
      - police_department
      - driver_gender
      - driver_age_raw
      - driver_age
      - driver_race_raw
      - driver_race
      - violation_raw
      - violation
      - search_conducted
      - search_type_raw
      - search_type
      - contraband_found
      - stop_outcome
      - is_arrested
      - ethnicity

Overwriting store.yaml
